mounting colab and imports for the notebook.
mounting colab is not necessary for kaggle.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import gc


In [0]:
path = 'drive/My Drive/DLWS/Assignment2/elo/'

**contest kernel**

we didnt present any EDA, all the information we have about how the data look and behave we learnd from severl EDA kernels that are public on the contest page.

This function is used to reduce memory use to load the history data which crashes kaggle and colab. it is also taken from a public kaggler notebook.

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Train and Test preprocess

In [81]:
train = pd.read_csv(path + 'train.csv', parse_dates=['first_active_month'])
test = pd.read_csv(path + 'test.csv', parse_dates=['first_active_month'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123623 entries, 0 to 123622
Data columns (total 5 columns):
first_active_month    123622 non-null datetime64[ns]
card_id               123623 non-null object
feature_1             123623 non-null int64
feature_2             123623 non-null int64
feature_3             123623 non-null int64
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 4.7+ MB


In [0]:
for feature in ['feature_1','feature_2','feature_3']:
  train[feature] = train[feature].astype('category')
  test[feature] = test[feature].astype('category')

In [83]:
test['first_active_month'].fillna(test['first_active_month'].value_counts().idxmax(), inplace = True) 


first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64

In [84]:
max_date = train['first_active_month'].dt.date.max()

def date_parse(df):
  date_parts = ['year', 'month' , 'weekday']
  for part in date_parts:
    date_part_col = 'first_active_month_' + part
    df[date_part_col] = getattr(df['first_active_month'].dt, part).astype(int)
  df['elasped_time'] = (max_date - df['first_active_month'].dt.date).dt.days
  return df

train = date_parse(train)
test = date_parse(test)

train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_active_month_year,first_active_month_month,first_active_month_weekday,elasped_time
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,2017,6,3,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,2017,1,6,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,2016,8,0,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,2017,9,4,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,2017,11,2,92


# History and new merchents transactions preprocess

In [0]:
his_trans = pd.read_csv(path + 'historical_transactions.csv', parse_dates=['purchase_date'])
n_merch_trans = pd.read_csv(path + 'new_merchant_transactions.csv', parse_dates=['purchase_date'])

In [0]:
def history_preprocess(df):
  df['authorized_flag'] = df['authorized_flag'].apply(lambda x: 1 if x=='Y' else 0) 
  df['category_1'] = df['category_1'].apply(lambda x: 0 if x=='Y' else 1)
  df['category_2'] = df['category_2'].fillna(0)
  df['category_3'] = df['category_3'].map({'A': 0, 'B': 1, 'C': 2})
  df['category_3'] = df['category_3'].fillna(3)
  df['category_3'] = df['category_3'].astype('int')

  df['installments'] = df['installments'].astype('category')
  return df

his_trans = history_preprocess(his_trans)
n_merch_trans = history_preprocess(n_merch_trans)

# History and transactions feature engineering

In [0]:
def aggregate_historical_transactions(trans, prefix):
    trans['purchase_month'] = trans['purchase_date'].dt.month
    trans['year'] = trans['purchase_date'].dt.year
    trans['dayofweek'] = trans['purchase_date'].dt.dayofweek
    trans['installments'] = trans['installments'].astype(int)
    trans['month_diff'] = ((datetime.today() - trans['purchase_date']).dt.days)//30
    trans['month_diff'] += trans['month_lag']
    trans['installments'] = trans['installments'].astype(int)
    trans = pd.get_dummies(trans, columns=['category_2', 'category_3'])
    agg_func = {
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean', 'sum'],
        'category_2_2.0': ['mean', 'sum'],
        'category_2_3.0': ['mean', 'sum'],
        'category_2_4.0': ['mean', 'sum'],
        'category_2_5.0': ['mean', 'sum'],
        'category_3_1': ['sum', 'mean'],
        'category_3_2': ['sum', 'mean'],
        'category_3_3': ['sum', 'mean'],
        'merchant_id': ['nunique'],
        'purchase_amount': ['sum', 'mean'],
        'installments': ['sum', 'mean'],
        'purchase_month': ['mean', 'max', 'min',],
        'purchase_date': ['max', 'min'],
        'month_lag': ['min', 'max'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'subsector_id': ['nunique'],
        'city_id': ['nunique'],
    }
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (trans.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')
    return agg_trans

In [88]:
n_merch_trans = aggregate_historical_transactions(n_merch_trans,'new')

h1
h2
h3
h4


In [0]:
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_lag'])
    history['installments'] = history['installments'].astype(int)
    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group

final_group = aggregate_per_month(his_trans) 

In [91]:
historical_transactions = reduce_mem_usage(his_trans)
his_trans = aggregate_historical_transactions(historical_transactions, prefix='hist_')

Mem. usage decreased to 1166.08 Mb (62.5% reduction)
h1
h2
h3
h4


In [92]:
his_trans.head()

,card_id,hist_transactions_count,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_1.0_sum,hist_category_2_2.0_mean,hist_category_2_2.0_sum,hist_category_2_3.0_mean,hist_category_2_3.0_sum,...,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_min,hist_month_lag_max,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_subsector_id_nunique,hist_city_id_nunique
0,C_ID_00007093c1,149,121.0,0.812081,0.000000,0.0,0.000000,0.0,0.805369,120.0,...,12,1,2018-02-27 05:14:57,2017-02-14 14:00:43,-12,0,18,3,13,4
1,C_ID_0001238066,123,121.0,0.983740,0.772358,95.0,0.000000,0.0,0.000000,0.0,...,12,1,2018-02-27 16:18:59,2017-09-28 22:25:14,-5,0,29,6,17,18
2,C_ID_0001506ef0,66,66.0,1.000000,0.030303,2.0,0.000000,0.0,0.969697,64.0,...,12,1,2018-02-17 12:33:56,2017-01-14 16:16:01,-13,0,19,2,12,3
3,C_ID_0001793786,216,214.0,0.990741,0.050926,11.0,0.351852,76.0,0.069444,15.0,...,10,1,2017-10-31 20:20:18,2017-01-21 10:15:21,-9,0,48,4,24,10
4,C_ID_000183fdda,144,140.0,0.972222,0.048611,7.0,0.006944,1.0,0.909722,131.0,...,12,1,2018-02-25 20:57:08,2017-08-07 09:49:14,-6,0,36,7,21,9


In [93]:
train = pd.merge(train, his_trans, on='card_id', how='left')
test = pd.merge(test, his_trans, on='card_id', how='left')
del his_trans

ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

...

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


In [94]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_active_month_year,first_active_month_month,first_active_month_weekday,elasped_time,...,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_min,hist_month_lag_max,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_subsector_id_nunique,hist_city_id_nunique
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,2017,6,3,245,...,12,1,2018-02-25 09:31:15,2017-06-27 14:18:08,-8,0,41,3,21,7
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,2017,1,6,396,...,12,1,2018-01-31 22:31:09,2017-01-06 16:29:42,-12,0,57,3,24,9
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,2016,8,0,549,...,12,1,2018-02-27 19:08:25,2017-01-11 08:21:22,-13,0,8,2,7,5
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,2017,9,4,153,...,12,1,2018-02-28 11:44:40,2017-09-26 16:22:21,-5,0,25,5,13,7
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,2017,11,2,92,...,12,1,2018-02-28 20:40:41,2017-11-12 00:00:00,-3,0,26,6,17,6


In [95]:
del historical_transactions
gc.collect()

1249

In [96]:
train = pd.merge(train, n_merch_trans, on='card_id', how='left')
test = pd.merge(test, n_merch_trans, on='card_id', how='left')
train.head()

ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

...

ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_active_month_year,first_active_month_month,first_active_month_weekday,elasped_time,...,newpurchase_month_max,newpurchase_month_min,newpurchase_date_max,newpurchase_date_min,newmonth_lag_min,newmonth_lag_max,newmerchant_category_id_nunique,newstate_id_nunique,newsubsector_id_nunique,newcity_id_nunique
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,2017,6,3,245,...,4.0,3.0,2018-04-29 11:23:05,2018-03-05 14:04:36,1.0,2.0,14.0,1.0,10.0,3.0
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,2017,1,6,396,...,3.0,2.0,2018-03-30 06:48:26,2018-02-01 17:07:54,1.0,2.0,5.0,1.0,4.0,1.0
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,2016,8,0,549,...,4.0,4.0,2018-04-28 17:43:11,2018-04-28 17:43:11,2.0,2.0,1.0,1.0,1.0,1.0
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,2017,9,4,153,...,4.0,3.0,2018-04-18 11:00:11,2018-03-07 11:55:06,1.0,2.0,6.0,2.0,5.0,2.0
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,2017,11,2,92,...,4.0,3.0,2018-04-28 18:50:25,2018-03-02 11:55:43,1.0,2.0,17.0,5.0,10.0,5.0


In [97]:
del n_merch_trans
gc.collect()

219

In [98]:
train = pd.merge(train, final_group, on='card_id')
test = pd.merge(test, final_group, on='card_id')
gc.collect()
del final_group

ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

...

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: ignored

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


In [99]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_active_month_year,first_active_month_month,first_active_month_weekday,elasped_time,...,installments_sum_mean,installments_sum_std,installments_mean_mean,installments_mean_std,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,2017,6,3,245,...,0.444444,0.726483,0.018182,0.032221,0.000000,0.000000,0.333333,0.500000,0.074290,0.117067
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,2017,1,6,396,...,41.769231,14.341977,1.713279,0.546685,0.692308,0.751068,6.307692,2.780426,1.478938,0.895708
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,2016,8,0,549,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,2017,9,4,153,...,14.000000,11.523888,1.044819,0.215637,0.333333,1.032796,2.000000,1.095445,0.518905,0.483284
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,2017,11,2,92,...,45.500000,23.979158,1.369969,0.597277,1.000000,0.000000,4.500000,5.066228,1.083005,1.664428


# Preparing the Data for the model

In [0]:
for col in train.columns:
    if train[col].isna().any():
        train[col] = train[col].fillna(0)

In [0]:
for col in test.columns:
    if test[col].isna().any():
        test[col] = test[col].fillna(0)

In [0]:
y = train['target']

In [0]:
col_to_drop = ['first_active_month', 'card_id', 'target','hist_purchase_date_max', 'hist_purchase_date_min', 'newpurchase_date_max', 'newpurchase_date_min']
for col in col_to_drop:
    if col in train.columns:
        train.drop([col], axis=1, inplace=True)
    if col in test.columns:
        test.drop([col], axis=1, inplace=True)

In [0]:
train['feature_3'] = train['feature_3'].astype(int)
test['feature_3'] = test['feature_3'].astype(int)

In [0]:
from sklearn.preprocessing import LabelEncoder
categorical_feats = ['feature_1', 'feature_2']
for col in categorical_feats:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
    test[col] = lbl.transform(list(test[col].values.astype('str')))

In [0]:
X = train
X_test = test

# Predictaion and models

Using kfold validation

In [0]:
from sklearn.model_selection import KFold
n_fold = 2
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)

using lgb model kfold validation for predication

In [0]:
import lightgbm as lgb
import time
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import mean_squared_error
def train_model(X=X, X_test=X_test, y=y, params=None, folds=folds, model_type='lgb'):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse',
                    verbose=1000, early_stopping_rounds=200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_squared_error(y_valid, y_pred_valid) ** 0.5)
        
        prediction += y_pred 
            
    prediction /= n_fold
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    return oof, prediction


params for lgb - using try and error gridsearch we ran out of kernel and other public kernels.

In [0]:
params = {'num_leaves': 54,
         'min_data_in_leaf': 79,
         'objective': 'regression',
         'max_depth': 7,
         'learning_rate': 0.018545526395058548,
         "boosting": "gbdt",
         "feature_fraction": 0.8354507676881442,
         "bagging_freq": 3,
         "bagging_fraction": 0.8126672064208567,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         'min_child_weight': 5.343384366323818,
         'reg_alpha': 1.1302650970728192,
         'reg_lambda': 0.3603427518866501,
         'subsample': 0.8767547959893627,}

In [143]:
oof_lgb, prediction_lgb = train_model(params=params, model_type='lgb')

Fold 0 started at Sat Dec 22 19:29:24 2018
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[740]	training's rmse: 3.447	valid_1's rmse: 3.73263
Fold 1 started at Sat Dec 22 19:30:46 2018
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[662]	training's rmse: 3.47633	valid_1's rmse: 3.65562
Fold 2 started at Sat Dec 22 19:32:03 2018
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[732]	training's rmse: 3.45278	valid_1's rmse: 3.70344
Fold 3 started at Sat Dec 22 19:33:26 2018
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[444]	training's rmse: 3.52365	valid_1's rmse: 3.67968
Fold 4 started at Sat Dec 22 19:34:25 2018
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[497]	training's rmse: 3.52065	valid_1's rmse: 3.64126
CV mean score: 3.6825,

Creating submission file from the model predictions

In [124]:
submission = pd.read_csv(path + 'sample_submission.csv')
submission['target'] = prediction_lgb
submission.to_csv(path + 'lgb_d.csv', index=False)
submission.head()

,card_id,target
0,C_ID_0ab67a22ab,-3.063205
1,C_ID_130fd0cbdd,-0.596065
2,C_ID_b709037bc5,-0.488714
3,C_ID_d27d835a9f,-0.284089
4,C_ID_2b5e3df5c2,-0.671911


Trying to use ANN to imprive results - did not work well.

In [0]:
from keras  import *
inp = Input(shape=(99,))
z = Dense(2048, activation='relu')(inp)
z = Dense(1024, activation='relu')(z)
z = Dense(512, activation='sigmoid')(z)
z = Dense(256, activation='relu')(z)
z = Dense(1, activation='relu')(z)

model = Model(inputs = inp, outputs= z )
model.compile(loss = 'mse', optimizer = 'RMSProp')

In [137]:
history = model.fit(x=X, y=y, batch_size=64, epochs=5)

Epoch 1/5
201917/201917 [==============================] - 27s 132us/step - loss: 14.9812
Epoch 2/5
201917/201917 [==============================] - 26s 129us/step - loss: 14.9812
Epoch 3/5
201917/201917 [==============================] - 26s 130us/step - loss: 14.9812
Epoch 4/5
 80384/201917 [==========>...................] - ETA: 15s - loss: 15.2742

KeyboardInterrupt: ignored